# Merlot Reserve Pretraining loss 


```text
L_text  = Text corruption matching.  -- via span prediction..? 
L_audio = Audio corruption matching. -- span prediction
L_frame = Img to text matching.      -- matching 
```

What are these? "Sources"? 
['text2audio', 'audio2text', 'random_text']:

In [3]:
#### LOSS is here
# Yes, but a little hard to read. They clearly compute 3 losses. text2audio, audio2text, and random_text.

# jnp == import jax.numpy as jnp

preds = {'text_preds': {'logits': [1,2,3]},
        'audio_preds': 'audio_preds', 
        'audio_preds2': 'audio_preds2'
        }

def loss_fn_given_preds(preds):
    loss_info = {}

    if 'text_preds' in preds:
        # Special-case of mask LM loss
        text_preds = preds.pop('text_preds')
        logits = text_preds['logits']
        
        # get predictions from logits over vocab(?) or what's num_classes?
        labels = jax.nn.one_hot(text_preds['labels'], num_classes=logits.shape[1], dtype=logits.dtype)
        logprobs = jax.nn.log_softmax(logits, axis=-1)
        mask = (text_preds['labels'] != 0).astype(logits.dtype)

        # Kastan -- what is mask.sum() doing?
        loss_info['audio2text'] = -(jnp.sum(logprobs * labels, axis=-1) * mask).sum() / mask.sum()

    for c_type, c_dict in preds.items():
        numer_logits = (c_dict['x'] * c_dict['y']).sum(-1)
        loss_info[c_type] = 0.0

        if '_sources' in c_dict:
            for k in ['text2audio', 'audio2text', 'random_text']:
                loss_info[f'_{c_type}_from_{k}'] = 0.0
        # For both directions (average the result)
        for k1, k2 in ['xy', 'yx']:
            x = c_dict[k1]
            y = c_dict[k2]

            # Add in extra things that are only valid as targets
            if f'{k2}_extra' in c_dict:
                y = jnp.concatenate([y, c_dict[f'{k2}_extra']])
            y_allgather = jax.lax.all_gather(y, 'batch').reshape(-1, x.shape[-1])

            print(f"{c_type} {k1}->{k2} dot product sim:  shaped [{x.shape}] -> [{y_allgather.shape}", flush=True)
            denom_logits = jnp.einsum('lh,vh->lv', x, y_allgather)
            denom_lse = jax.nn.logsumexp(denom_logits.astype(jnp.float32), axis=-1)
            loss_info[c_type] += (denom_lse - numer_logits).mean() / 2.0
            if '_sources' in c_dict:
                for i, type_i in enumerate(['text2audio', 'audio2text', 'random_text']):
                    does_match = (c_dict['_sources'] == i).astype(jnp.float32)
                    loss_match = ((denom_lse - numer_logits) * does_match).sum() / (does_match.sum() + 1e-5)
                    loss_info[f'_{c_type}_from_{type_i}'] += loss_match / 2.0

    loss = sum([v for k, v in loss_info.items() if not k.startswith('_')])
    return loss, loss_info



NameError: name 'jax' is not defined

In [ ]:
def train_step(state: train_state.TrainState, batch, use_bfloat16_grads=True):
    """
    Note: we'll compile this with pmap so no need to jit
    :param state:
    :param batch:
    :param use_bfloat16_grads: Whether to use bfloat16 for storing grads. I think it is probably OK considering
                               momentum is bfloat16 anyways. i'm just going to cast down (rounding down here rather
                               than to nearest or anything)
    :return:
    """

    def _loss_fn(params):
        return loss_fn_given_preds(state.apply_fn({'params': params}, batch))

    grad_fn = jax.value_and_grad(_loss_fn, has_aux=True)

    params = state.params
    if use_bfloat16_grads:
        params = f32_to_bf16(state.params)

    (loss, loss_info), grads = grad_fn(params)

    grads = jax.tree_map(lambda x: jnp.nan_to_num(x, copy=False), grads)
    grads = jax.lax.pmean(grads, axis_name='batch')

    # Cast up grads here (after the pmean) which reduces bandwidth maybe
    if use_bfloat16_grads:
        grads = bf16_to_f32(grads)

    # Average metrics over all replicas (maybe this isn't a great idea, idk)
    loss_info = jax.lax.pmean(loss_info, axis_name='batch')
    loss_info = bf16_to_f32(loss_info)

    new_state = state.apply_gradients(grads=grads)
    return new_state, loss_info